# Second Classwork – Data Mining (IDM 2025/2026)

In questo secondo classwork l’obiettivo è costruire un **classificatore multi-classe** capace di predire, dato un nuovo paziente, una delle tre classi:

- **ASD**: bambini con diagnosi di autismo  
- **GDD**: bambini con global developmental delay  
- **Controls**: bambini con sviluppo tipico  

Il dataset è fornito in formato **Excel** con tre fogli (ASD, GDD, Controls/Controlli). Ogni paziente è descritto da feature discrete con valori in {0, 1, 2}. Prima dell’addestramento va effettuata una pulizia accurata:

- rimuovere le feature **non rilevanti**: `Età cronologica (mesi)`, `Scala B`, `Scala D`, `TOT.`, `Score di rischio`;
- escludere i pazienti con **Età equivalente < 12 mesi** (per evitare rumore dovuto a feature “defaultate” a 0 nei primi mesi).

Il notebook segue i task richiesti:

1. **PCA 2D** e scatter plot dei pazienti;
2. **Split** train/test;
3. Training di più classificatori (Decision Tree, Random Forest, SVC, K-NN) con selezione del migliore tramite **GridSearchCV con 5-fold cross validation**;
4. Uso di **bagging** e **boosting** per migliorare le prestazioni.

L’implementazione operativa delle pipeline è incapsulata in moduli `src/phase*.py` basati su TRACCIA; questo notebook si limita ad **orchestrare** le fasi e a visualizzare i risultati.


## Setup ambiente

In questa cella installiamo le dipendenze del progetto (incluse `TRACCIA` e le librerie di machine learning).
Se stai lavorando in un ambiente notebook, potrebbe essere necessario **riavviare il kernel** dopo l’installazione.


In [1]:
pip install -r requirements.txt # type: ignore

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emanuelegaliano/TRACCIA.git to /tmp/pip-req-build-7raxg_49
  Running command git clone --filter=blob:none --quiet https://github.com/emanuelegaliano/TRACCIA.git /tmp/pip-req-build-7raxg_49
  Resolved https://github.com/emanuelegaliano/TRACCIA.git to commit 4dd9ba02a86286c7e28196f8a24696279416caf0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


## Fase 0 — Pulizia del dataset (Excel → CSV clean)

Questa fase carica `data/dataset_raw.xlsx` (tre fogli: ASD, GDD, Controlli/Controls) e produce un dataset pulito:

- unisce i fogli e crea la colonna target `class`;
- rimuove le 5 feature indicate nell’assignment;
- filtra i pazienti con `Età equivalente < 12`;
- codifica `Sesso` in forma numerica;
- gestisce i duplicati di colonne (es. `B10` ripetuta) rinominandoli in modo univoco (`B10_y1`, `B10_y2`, …);
- esporta **un solo file**: `data/phase0_clean.csv`.


In [2]:
from pathlib import Path
from src.phase0_clean import Phase0Config, run_phase0

cfg = Phase0Config(
    data_dir=Path("data"),
    input_xlsx="dataset_raw.xlsx",
    output_clean_csv="phase0_clean.csv",
    missing_strategy="drop_rows",  # oppure "impute_median"
)

out = run_phase0(cfg, trace=True)
print("Phase0 output:", out)

[TRACCIA] Trail 'phase0_clean' starting
[TRACCIA] -> LoadXLSX
[TRACCIA] -> NormalizeAndRename
[TRACCIA] -> DropAssignmentColumns
[TRACCIA] -> FilterAgeEquivalent
[TRACCIA] -> EncodeSex
[TRACCIA] -> CoerceNumericAndMissing
[TRACCIA] -> ExportCleanCSV
[TRACCIA] Trail 'phase0_clean' finished (handlers=['LoadXLSX', 'NormalizeAndRename', 'DropAssignmentColumns', 'FilterAgeEquivalent', 'EncodeSex', 'CoerceNumericAndMissing', 'ExportCleanCSV'])
Phase0 output: data/phase0_clean.csv


## Fase 1 — PCA a 2 componenti e scatter plot

In questa fase carichiamo il dataset pulito e:

- separiamo feature `X` e target `y`;
- (opzionale ma consigliato) applichiamo **StandardScaler** prima della PCA;
- riduciamo a 2 dimensioni con **PCA(n_components=2)**;
- generiamo lo **scatter plot 2D** colorato per classe e salviamo l’immagine in `reports/phase1_pca2d.png`.


In [3]:
from pathlib import Path
from src.phase1_pca2d import Phase1Config, run_phase1

cfg = Phase1Config(
    data_dir=Path("data"),
    reports_dir=Path("reports"),
    input_clean_csv="phase0_clean.csv",
    output_png="phase1_pca2d.png",
    scale_before_pca=True,
)

out = run_phase1(cfg, trace=True)
print("Phase1 output:", out)

[TRACCIA] Trail 'phase1_pca2d' starting
[TRACCIA] -> LoadClean
[TRACCIA] -> BuildXY
[TRACCIA] -> ComputePCA2D
[TRACCIA] -> PlotAndSavePNG
[TRACCIA] Trail 'phase1_pca2d' finished (handlers=['LoadClean', 'BuildXY', 'ComputePCA2D', 'PlotAndSavePNG'])
Phase1 output: reports/phase1_pca2d.png


## Fase 2 — Train/Test split + GridSearchCV (5-fold)

Questa fase implementa il Task principale di classificazione:

- split stratificato train/test;
- per ciascun modello (Decision Tree, Random Forest, SVC, K-NN) eseguiamo una **GridSearchCV con 5-fold cross validation** per selezionare iperparametri che riducano l’overfitting;
- valutiamo il modello migliore su test con metriche (accuracy e F1-macro);
- esportiamo **un solo file**: `reports/phase2_gridsearch.json` con best params, score CV e metriche su test.


In [4]:
from pathlib import Path
from src.phase2_gridsearch import Phase2Config, run_phase2

cfg = Phase2Config(
    data_dir=Path("data"),
    reports_dir=Path("reports"),
    scoring="f1_macro",
)

out = run_phase2(cfg, trace=True)
print("Phase2 output:", out)

[TRACCIA] Trail 'phase2_gridsearch' starting
[TRACCIA] -> LoadClean
[TRACCIA] -> BuildXY
[TRACCIA] -> SplitTrainTest
[TRACCIA] -> GridSearchAllModels
[TRACCIA] -> ExportJSON
[TRACCIA] Trail 'phase2_gridsearch' finished (handlers=['LoadClean', 'BuildXY', 'SplitTrainTest', 'GridSearchAllModels', 'ExportJSON'])
Phase2 output: reports/phase2_gridsearch.json


## Report risultati Fase 2 — tabella comparativa

Qui leggiamo `reports/phase2_gridsearch.json` e costruiamo una tabella comparativa con:

- miglior score in cross-validation (F1 macro),
- accuracy e F1 macro sul test set,
- iperparametri migliori trovati per ciascun modello.


In [5]:
import json
import pandas as pd
from pathlib import Path

json_path = Path("reports/phase2_gridsearch.json")

with json_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

results = data["results"]

rows = []
for model_name, res in results.items():
    rows.append({
        "Model": model_name,
        "CV Best Score (f1_macro)": round(res["cv_best_score"], 4),
        "Test Accuracy": round(res["accuracy_test"], 4),
        "Test F1 Macro": round(res["f1_macro_test"], 4),
        "Best Params": res["best_params"],
    })

df_results = pd.DataFrame(rows).sort_values("Test F1 Macro", ascending=False)
df_results

,Model,CV Best Score (f1_macro),Test Accuracy,Test F1 Macro,Best Params
0,DecisionTree,0.6062,0.725,0.7230,"{'max_depth': 5, 'min_samples_split': 10}"
2,SVC,0.6972,0.650,0.6518,"{'svc__C': 0.1, 'svc__kernel': 'linear'}"
3,KNN,0.6024,0.625,0.6257,"{'knn__n_neighbors': 5, 'knn__weights': 'dista..."
1,RandomForest,0.7230,0.625,0.6186,"{'max_depth': 10, 'n_estimators': 300}"


## Report Fase 2 — confusion matrix di un modello

In questa cella ispezioniamo la **confusion matrix** di uno specifico modello (impostato in `model_name`).
La matrice permette di capire dove avvengono gli errori di classificazione tra ASD, GDD e Controls.


In [6]:
model_name = "DecisionTree"

cm = results[model_name]["confusion_matrix"]
print("Confusion Matrix:")
print(pd.DataFrame(cm))

Confusion Matrix:
    0  1   2
0  11  1   3
1   0  7   4
2   1  2  11


## Fase 3 — Ensemble learning: Bagging e Boosting

In questa fase applichiamo i concetti di ensemble richiesti:

- **Bagging**: più Decision Tree addestrati su bootstrap / sotto-campionamenti per ridurre la varianza;
- **Boosting (AdaBoost)**: sequenza di weak learners (stump) che si concentrano progressivamente sugli errori.

Valutiamo entrambi su test set e salviamo **un solo file**: `reports/phase3_ensembles.json`.


In [7]:
from pathlib import Path
import json
import pandas as pd

from src.phase3_ensembles import Phase3Config, run_phase3

cfg = Phase3Config(
    data_dir=Path("data"),
    reports_dir=Path("reports"),
    input_clean_csv="phase0_clean.csv",
    output_json="phase3_ensembles.json",
    test_size=0.2,
    random_state=42,
)

out = run_phase3(cfg, trace=True)
print("Phase3 output:", out)

with open(out, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for k, r in data["results"].items():
    rows.append({
        "Ensemble": r["model_name"],
        "Test Accuracy": round(r["accuracy_test"], 4),
        "Test F1 Macro": round(r["f1_macro_test"], 4),
        "Params": r["params"],
    })

pd.DataFrame(rows).sort_values("Test F1 Macro", ascending=False)

[TRACCIA] Trail 'phase3_ensembles' starting
[TRACCIA] -> LoadClean
[TRACCIA] -> BuildXY
[TRACCIA] -> SplitTrainTest
[TRACCIA] -> TrainBagging
[TRACCIA] -> TrainBoosting
[TRACCIA] -> ExportJSON
[TRACCIA] Trail 'phase3_ensembles' finished (handlers=['LoadClean', 'BuildXY', 'SplitTrainTest', 'TrainBagging', 'TrainBoosting', 'ExportJSON'])
Phase3 output: reports/phase3_ensembles.json


,Ensemble,Test Accuracy,Test F1 Macro,Params
0,Bagging(DecisionTree),0.650,0.6424,"{'n_estimators': 300, 'max_samples': 0.8, 'max..."
1,AdaBoost(DecisionTree),0.575,0.5637,"{'n_estimators': 300, 'learning_rate': 0.5, 'b..."


## Report risultati Fase 3 — confronto + dettagli per classe

Qui produciamo un report leggibile per gli ensemble:

- tabella comparativa (accuracy e F1 macro sul test);
- confusion matrix per ciascun ensemble;
- classification report con precision/recall/F1 per classe.


In [8]:
import json
import pandas as pd
from pathlib import Path

# ---- Carica JSON ----
json_path = Path("reports/phase3_ensembles.json")

with json_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

results = data["results"]

# ---- Tabella comparativa principale ----
rows = []
for name, res in results.items():
    rows.append({
        "Model": res["model_name"],
        "Accuracy (test)": round(res["accuracy_test"], 4),
        "F1 Macro (test)": round(res["f1_macro_test"], 4),
    })

df_summary = pd.DataFrame(rows).sort_values("F1 Macro (test)", ascending=False)

print("=== Phase 3 - Ensemble Comparison ===")
display(df_summary)

# ---- Miglior modello ----
best_model = df_summary.iloc[0]["Model"]
print(f"\nBest Ensemble (by F1 Macro): {best_model}\n")

# ---- Confusion Matrix + Metriche dettagliate ----
for name, res in results.items():
    print("="*60)
    print(f"Model: {res['model_name']}")
    print("-"*60)

    # Confusion Matrix
    labels = res["labels"]
    cm = pd.DataFrame(res["confusion_matrix"], index=labels, columns=labels)
    print("Confusion Matrix:")
    display(cm)

    # Metriche per classe
    report_df = pd.DataFrame(res["classification_report"]).T
    print("Classification Report:")
    display(report_df.round(4))

    print("\n")

=== Phase 3 - Ensemble Comparison ===


,Model,Accuracy (test),F1 Macro (test)
0,Bagging(DecisionTree),0.650,0.6424
1,AdaBoost(DecisionTree),0.575,0.5637



Best Ensemble (by F1 Macro): Bagging(DecisionTree)

Model: Bagging(DecisionTree)
------------------------------------------------------------
Confusion Matrix:


,ASD,Controls,GDD
ASD,12,1,2
Controls,0,7,4
GDD,3,4,7


Classification Report:


,precision,recall,f1-score,support
ASD,0.8000,0.8000,0.8000,15.00
Controls,0.5833,0.6364,0.6087,11.00
GDD,0.5385,0.5000,0.5185,14.00
accuracy,0.6500,0.6500,0.6500,0.65
macro avg,0.6406,0.6455,0.6424,40.00
weighted avg,0.6489,0.6500,0.6489,40.00




Model: AdaBoost(DecisionTree)
------------------------------------------------------------
Confusion Matrix:


,ASD,Controls,GDD
ASD,10,1,4
Controls,0,4,7
GDD,2,3,9


Classification Report:


,precision,recall,f1-score,support
ASD,0.8333,0.6667,0.7407,15.000
Controls,0.5000,0.3636,0.4211,11.000
GDD,0.4500,0.6429,0.5294,14.000
accuracy,0.5750,0.5750,0.5750,0.575
macro avg,0.5944,0.5577,0.5637,40.000
weighted avg,0.6075,0.5750,0.5789,40.000
